##### library

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
from datetime import datetime, timedelta
import msoffcrypto
from io import BytesIO
import os
from openpyxl import Workbook
import re

##### dataframe

In [30]:
today = datetime.today()
today = today.strftime('%y%m%d')

In [31]:
#cafe24

df_cfv = None
df_cfa = None

#path
path_cf = f'.\data\{today}_cf.csv'
path_cfv = f'.\data\{today}_cfv.csv'
path_cfa = f'.\data\{today}_cfa.csv'

#myhabit
df_cf = pd.read_csv(path_cf)


#vitalfit
if os.path.exists(path_cfv):
    df_cfv = pd.read_csv(path_cfv)
else:
    pass

#arhee
if os.path.exists(path_cfa):
    df_cfa = pd.read_csv(path_cfa)
else:
    pass

<string>:7: SyntaxWarning: invalid escape sequence '\{'
<string>:8: SyntaxWarning: invalid escape sequence '\{'
<string>:9: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\{'
<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\{'
<>:9: SyntaxWarning: invalid escape sequence '\d'
<string>:7: SyntaxWarning: invalid escape sequence '\{'
<string>:8: SyntaxWarning: invalid escape sequence '\{'
<string>:9: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\{'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\{'
<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\{'
<>:9: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_7840\387831278.py:7: 

In [32]:
#smartstore

#path
path_ss = f'.\data\{today}_ss.xlsx'

#password
password_ss = '1945'

with open(path_ss, 'rb') as f:
    file = msoffcrypto.OfficeFile(f)
    file.load_key(password=password_ss)

    decrypted = BytesIO()
    file.decrypt(decrypted)

    df_ss = pd.read_excel(decrypted, engine='openpyxl', header = 1)

<string>:4: SyntaxWarning: invalid escape sequence '\{'
<>:4: SyntaxWarning: invalid escape sequence '\{'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<string>:4: SyntaxWarning: invalid escape sequence '\{'
<>:4: SyntaxWarning: invalid escape sequence '\{'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_7840\825252997.py:4: SyntaxWarning: invalid escape sequence '\{'
  path_ss = f'.\data\{today}_ss.xlsx'
C:\Users\user\AppData\Local\Temp\ipykernel_7840\825252997.py:4: SyntaxWarning: invalid escape sequence '\d'
  path_ss = f'.\data\{today}_ss.xlsx'


In [33]:
#coupang

# path
directory = '.\data'

dataframes = {}

for filename in os.listdir(directory):
    if filename.endswith('_cp.xlsx'):

        match = re.search(r'(\d+)_cp\.xlsx', filename)
        if match:
            date = match.group(1)

            filepath = os.path.join(directory, filename)

            df = pd.read_excel(filepath, engine='openpyxl')

            dataframes[f'df_cp{date}'] = df

for key, df in dataframes.items():
    date = key[5:]
    df['주문일시'] = date

df_cp = pd.concat(dataframes.values(), ignore_index=True)

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_7840\1695998503.py:4: SyntaxWarning: invalid escape sequence '\d'
  directory = '.\data'
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


##### cafe24

In [34]:
#myhabit
df_cf = df_cf[~df_cf['주문 상태'].str.contains('취소')]
df_cf = df_cf[~df_cf['주문경로'].str.contains('쿠팡')]
df_cf = df_cf.drop(columns=['주문 상태'])
df_cf['주문경로'] = df_cf['주문경로'].replace(['모바일웹', '네이버 페이', 'PC쇼핑몰'], '자사몰')
df_cf['총 배송비 (첫 품목에만 표시)'] = df_cf['총 배송비 (첫 품목에만 표시)'].fillna(0)
df_cf['배송비 추가결제'] = df_cf['배송비 추가결제'].fillna(0)

df_cf['결제일시(입금확인일)'] = pd.to_datetime(df_cf['결제일시(입금확인일)']).dt.strftime('%Y-%m-%d')
df_cf = df_cf.sort_values(by='주문경로', ascending=False)
df_cf = df_cf.sort_values(by='결제일시(입금확인일)', ascending=True)

In [35]:
#vitalfit

if os.path.exists(path_cfv):
    try:
        df_cfv = pd.read_csv(path_cfv)
        if df_cfv is not None:
            # 데이터 정제
            df_cfv = df_cfv[~df_cfv['주문 상태'].str.contains('취소', na=False)]
            df_cfv = df_cfv[~df_cfv['주문경로'].str.contains('쿠팡', na=False)]
            df_cfv = df_cfv.drop(columns=['주문 상태'])
            df_cfv['주문경로'] = df_cfv['주문경로'].replace(['모바일웹', '네이버 페이', 'PC쇼핑몰'], '바이탈핏')
            df_cfv['총 배송비 (첫 품목에만 표시)'] = df_cfv['총 배송비 (첫 품목에만 표시)'].fillna(0)
            df_cfv['배송비 추가결제'] = df_cfv['배송비 추가결제'].fillna(0)
            df_cfv['결제일시(입금확인일)'] = pd.to_datetime(df_cfv['결제일시(입금확인일)']).dt.strftime('%Y-%m-%d')
            df_cfv = df_cfv.sort_values(by='주문경로', ascending=False)
            df_cfv = df_cfv.sort_values(by='결제일시(입금확인일)', ascending=True)
    except Exception as e:
        print("데이터 처리 중 오류 발생:", e)
else:
    print("경로에 파일이 없습니다:", path_cfv)

경로에 파일이 없습니다: .\data\240708_cfv.csv


In [36]:
#arhee

if os.path.exists(path_cfa):
    try:
        df_cfa = pd.read_csv(path_cfa)
        if df_cfa is not None:
            # 데이터 정제
            df_cfa = df_cfa[~df_cfa['주문 상태'].str.contains('취소', na=False)]
            df_cfa = df_cfa[~df_cfa['주문경로'].str.contains('쿠팡', na=False)]
            df_cfa = df_cfa.drop(columns=['주문 상태'])
            df_cfa['주문경로'] = df_cfa['주문경로'].replace(['모바일웹', '네이버 페이', 'PC쇼핑몰'], '아르히')
            df_cfa['총 배송비 (첫 품목에만 표시)'] = df_cfa['총 배송비 (첫 품목에만 표시)'].fillna(0)
            df_cfa['배송비 추가결제'] = df_cfa['배송비 추가결제'].fillna(0)
            df_cfa['결제일시(입금확인일)'] = pd.to_datetime(df_cfa['결제일시(입금확인일)']).dt.strftime('%Y-%m-%d')
            df_cfa = df_cfa.sort_values(by='주문경로', ascending=False)
            df_cfa = df_cfa.sort_values(by='결제일시(입금확인일)', ascending=True)
    except Exception as e:
        print("데이터 처리 중 오류 발생:", e)
else:
    print("경로에 파일이 없습니다:", path_cfa)

##### smartstore

In [37]:
df_ss2 = pd.DataFrame()
df_ss2['결제일시(입금확인일)'] = df_ss['결제일']
df_ss2['주문일시'] = df_ss['결제일']
df_ss2['주문일시'] = df_ss2['주문일시'].astype(str)
df_ss2['주문경로'] = '스마트스토어'
df_ss2['주문번호'] = df_ss['주문번호']
df_ss2['상품코드'] = df_ss['상품번호']
df_ss['옵션관리코드'].fillna(df_ss['상품번호'], inplace=True)
df_ss2['상품품목코드'] = df_ss['옵션관리코드']
df_ss2['상품품목코드'] = df_ss2['상품품목코드'].astype(int).astype(str)
df_ss2['수량'] = df_ss['수량']
df_ss2['상품구매금액'] = df_ss['상품별 총 주문금액']
df_ss2['총 배송비 (첫 품목에만 표시)'] = df_ss['배송비 합계']
df_ss2['배송비 추가결제'] = df_ss['배송비 합계']
df_ss2['상품품목코드'] = df_ss2['상품품목코드'].astype(str)
df_ss2['결제일시(입금확인일)'] = pd.to_datetime(df_ss2['결제일시(입금확인일)']).dt.strftime('%Y-%m-%d')
df_ss2 = df_ss2.sort_values(by='결제일시(입금확인일)', ascending=True)

C:\Users\user\AppData\Local\Temp\ipykernel_7840\3938513147.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ss['옵션관리코드'].fillna(df_ss['상품번호'], inplace=True)


##### coupang

In [38]:
df_cp = df_cp[df_cp['순 판매 금액(전체 거래 금액 - 취소 금액)'] > 0]
df_cp = df_cp[df_cp['노출상품ID'] != '합 계']
df_cp2 = pd.DataFrame()
df_cp2['결제일시(입금확인일)'] = df_cp['주문일시']
df_cp2['결제일시(입금확인일)'] = pd.to_datetime(df_cp['주문일시'], format='%y%m%d').dt.strftime('%Y-%m-%d')
df_cp2['주문일시'] = df_cp['주문일시']
df_cp2['주문일시'] = pd.to_datetime(df_cp['주문일시'], format='%y%m%d').dt.strftime('%Y-%m-%d')
df_cp2['주문경로'] = df_cp['상품타입'].replace('판매자배송', '쿠팡윙')
df_cp2['주문번호'] = df_cp['노출상품ID']
df_cp2['상품코드'] = df_cp['노출상품ID']
df_cp2['상품품목코드'] = df_cp['옵션ID']
df_cp2['수량'] = df_cp['순 판매 상품 수(전체 거래 상품 수 - 취소 상품 수)']
df_cp2['상품구매금액'] = df_cp['순 판매 금액(전체 거래 금액 - 취소 금액)']
df_cp2['총 배송비 (첫 품목에만 표시)'] = df_cp['카테고리']
df_cp2['총 배송비 (첫 품목에만 표시)'] = np.nan
df_cp2['배송비 추가결제'] = df_cp['카테고리']
df_cp2['배송비 추가결제'] = np.nan
df_cp2['상품품목코드'] = df_cp2['상품품목코드'].astype(int)
df_cp2 = df_cp2.sort_values(by=['주문경로', '결제일시(입금확인일)'], ascending=True)

##### concat

In [39]:
df_result = pd.concat([df_cf, df_cfv, df_cfa, df_ss2, df_cp2], ignore_index=True)
df_result['총 배송비 (첫 품목에만 표시)'] = df_result['총 배송비 (첫 품목에만 표시)'].fillna(0)
df_result['배송비 추가결제'] = df_result['배송비 추가결제'].fillna(0)
df_result = df_result.fillna('')
df_result['총 배송비 (첫 품목에만 표시)'] = df_result['총 배송비 (첫 품목에만 표시)'].astype(int)
df_result['배송비 추가결제'] = df_result['배송비 추가결제'].astype(int)
df_result['상품구매금액'] = df_result['상품구매금액'].astype(int)

##### download

In [40]:
file_name = f'{today}_result.xlsx'

with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    df_result.to_excel(writer, index=False)


    workbook = writer.book
    worksheet = writer.sheets['Sheet1']


    for cell in worksheet['G']:
        cell.number_format = '@'

In [45]:
save_path = f'.\data\result\{file_name}'
df_result.to_excel(save_path, index=False)

<string>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_7840\2740732905.py:1: SyntaxWarning: invalid escape sequence '\{'
  save_path = f'.\data\result\{file_name}'
C:\Users\user\AppData\Local\Temp\ipykernel_7840\2740732905.py:1: SyntaxWarning: invalid escape sequence '\d'
  save_path = f'.\data\result\{file_name}'


OSError: Cannot save file into a non-existent directory: 'dataesult'